In [27]:
import requests

API_KEY = "q4gWkY1EjdZ88xcASJHJRJvnOsMRvxMNDSWOokQF"

def search_food_usda(query):
    """
    Search for a food item using USDA FoodData Central API

    Returns:
        List of dicts with basic nutrition info
    """
    url = "https://api.nal.usda.gov/fdc/v1/foods/search"
    params = {
        "api_key": API_KEY,
        "query": query,
        "pageSize": 3  # Limit results
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print("Failed to connect to USDA API.")
        return []

    data = response.json()
    foods = data.get("foods", [])
    results = []

    for item in foods:
        food_data = {
            "description": item.get("description", "N/A"),
            "brand": item.get("brandOwner", "N/A"),
            "calories": None,
            "protein": None,
            "carbs": None,
            "fat": None
        }

        for nutrient in item.get("foodNutrients", []):
            name = nutrient.get("nutrientName", "").lower()
            value = nutrient.get("value", 0)

            if "energy" in name:
                food_data["calories"] = value
            elif "protein" in name:
                food_data["protein"] = value
            elif "carbohydrate" in name:
                food_data["carbs"] = value
            elif "fat" in name:
                food_data["fat"] = value

        results.append(food_data)

    return results


In [28]:
import pandas as pd

def display_results_table(results):
    if not results:
        print("No results found.")
        return

    df = pd.DataFrame(results)
    display(df)


In [29]:
food = input("Enter a food name (in English): ")
results = search_food_usda(food)
display_results_table(results)


Enter a food name (in English): pizza


,description,brand,calories,protein,carbs,fat
0,PIZZA,The Kroger Co.,238,6.8,32.0,2.38
1,PIZZA,MANDIA,250,10.7,27.9,3.57
2,PIZZA,Casey's Bakery,199,10.6,19.2,4.96


In [30]:
def save_results_csv(results, filename="nutrition_results.csv"):
    df = pd.DataFrame(results)
    df.to_csv(filename, index=False)
    print(f"Results saved to {filename}")
save_results_csv(results)

Results saved to nutrition_results.csv
